# Reshape location extractions

This notebook is for reshaping the tables of indicators extracted at the provided locations of interest. Our collaborators have requested excel files containing the indicators values (for all decades and scenarios) using a separate document for each model. So we will start with creating one document for each model, and then the data will be split among tabs by location. 

Run imports:

In [1]:
from config import idx_decade_summary_dir, reshaped_decade_summary_dir, locations, models, out_dir
import numpy as np
import pandas as pd

### Tables by model

Iterate over locations and models and save the new tables in excel files by model, with tabs for each location:

In [2]:
%%time
for location in locations:
    loc_df = pd.read_csv(idx_decade_summary_dir.joinpath(f"decade_summaries_{location}.csv"))
    for model in models:
        model_df = loc_df.query(f"model == '{model}'").drop(columns="model")
        model_decade_fp = reshaped_decade_summary_dir.joinpath(f"decadal_cordex_indicator_summaries_{model}.xlsx")
        with pd.ExcelWriter(model_decade_fp, engine="openpyxl", mode="a", if_sheet_exists="replace") as decade_writer:
            model_df.to_excel(decade_writer, sheet_name=location, index=False)

CPU times: user 4min 55s, sys: 1.06 s, total: 4min 56s
Wall time: 4min 59s


### Barplots tables

Here we will combine the CSV files containing tables of model averages and min/max used for the barplots, found in `$OUTPUT_DIR/decadal_extractions`. We will combine them into a single excel file for ease of use.


In [2]:
%%time
out_fp = out_dir.joinpath("crossmodel_decadal_averages_extractions.xlsx")
out_fp.unlink(missing_ok=True)
with pd.ExcelWriter(out_fp, engine="openpyxl", mode="w") as decade_writer:
    for location in locations:
        # for location in locations:
        loc_df = pd.read_csv(idx_decade_summary_dir.joinpath(f"decade_summaries_{location}.csv"))
        loc_df = loc_df.rename(columns={"idx_var": "indicator"})
        xmodel_summary_df = (
            loc_df.groupby(["indicator", "scenario", "decade"])
            .agg({"mean": ["min", "mean", "max"]})
        ).droplevel(0, axis=1).reset_index().dropna()
        xmodel_summary_df["mean"] = xmodel_summary_df["mean"].round(1)
        xmodel_summary_df.to_excel(decade_writer, sheet_name=location, index=False)

CPU times: user 1.58 s, sys: 145 ms, total: 1.73 s
Wall time: 2 s


done